En este proceso se explica la forma en que pueden descargarse los datos disponibles en la [API](https://bchapi-am.developer.azure-api.net/) del Banco Central de Honduras (BCH), utilizando Python.

# Funciones


In [ ]:
import urllib.request, json
import pandas as pd
import plotly.express as px
import polars as pl

from IPython.display import Markdown
from tabulate import tabulate

In [ ]:
# class bchapi():
#     def __init__(self):
#         self.idslist = None
#         self.header = None

    # #get_info descarga toda la información de las series disponibles en base a la estructura del url ingresado
    # def get_info(self, url, hdr):
    #     self.header = hdr
    #     req = urllib.request.Request(url, headers=self.header)
    #     req.get_method = lambda: 'GET'
    #     response = urllib.request.urlopen(req)
    #     jsondata = response.read()
    #     bytes_data = jsondata
    #     data_str = bytes_data.decode('utf-8')
    #     data = json.loads(data_str)
    #     df = pd.read_json(json.dumps(data))
    #     self.idslist = df['Id'].unique()
    #     return df

    # #graph descarga los valores en formato dataframe de la serie seleccionada en base al id
    # def graph(self, id):
    #     id = self.idslist[idx] - 1
    #     url = f'https://bchapi-am.azure-api.net/api/v1/indicadores/{id}/cifras'
    #     df = self.get_info(url, self.header)
    #     ts = pd.DataFrame(
    #         data = df['Valor'].values, 
    #         index = pd.DatetimeIndex(df['Fecha']))
    #     fig = px.line(
    #         ts, 
    #         title = df['Descripcion'].unique()[0],)
    #     fig.update_layout(
    #         autosize=False,
    #         width=800,
    #         height=500,
    #         template = 'plotly_white',
    #         showlegend=False)
    #     fig.update_xaxes(title=None)
    #     fig.update_yaxes(title=None)
    #     fig.show()

In [ ]:
# A partir de código elaborado por Gabriela López:
# https://colab.research.google.com/drive/1R3zpo3L27ypjD7v_FO7GeFPQ_2GeEFKd?usp=sharing
def get_info(url, hdr):
    header = hdr
    req = urllib.request.Request(url, headers=header)
    req.get_method = lambda: 'GET'
    response = urllib.request.urlopen(req)
    jsondata = response.read()
    bytes_data = jsondata
    data_str = bytes_data.decode('utf-8')
    data = json.loads(data_str)
    df = pd.read_json(json.dumps(data))
    idslist = df['Id'].unique()
    return df

def get_df(id):
    url_0 = "https://bchapi-am.azure-api.net/api/v1/indicadores?formato=Json"
    url = f'https://bchapi-am.azure-api.net/api/v1/indicadores/{id}/cifras'
    all_variables = get_info(url_0, hdr)
    data = get_info(url, hdr)
    df = pd.DataFrame(
        data = data['Valor'].values, 
        index = pd.DatetimeIndex(data['Fecha']))
    df.columns = [all_variables["Descripcion"][id-1]]
    df = pl.from_pandas(df,include_index=True
        ).with_columns(
            pl.col("Fecha").cast(pl.Date))
    return df

def get_plot(id):
    url_0 = "https://bchapi-am.azure-api.net/api/v1/indicadores?formato=Json"
    url = f'https://bchapi-am.azure-api.net/api/v1/indicadores/{id}/cifras'
    all_variables = get_info(url_0, hdr)
    data = get_info(url, hdr)
    df = pd.DataFrame(
        data = data['Valor'].values, 
        index = pd.DatetimeIndex(data['Fecha']))
    fig = px.line(
        df, 
        # title = all_variables['Descripcion'].unique()[0],
        title = all_variables["Descripcion"][id-1]
        )
    fig.update_layout(
        autosize=False,
        width=800,
        height=500,
        template = 'plotly_white',
        showlegend=False)
    fig.update_xaxes(title=None)
    fig.update_yaxes(title=None)
    fig.show()

In [ ]:
def res_nivel_1():
    df = pl.DataFrame(df_all)
    variable_0 = "Niv_Gr_1"
    variable_1 = "Niv_Gr_2"
    variable_2 = "Niv_Gr_3"
    df = df.group_by(variable_0
        ).agg(
            pl.col(variable_0).len().alias("N_Variables"),
            pl.col(variable_1).n_unique().alias("N_Subgrupos")
        ).sort(["N_Variables","N_Subgrupos",],descending=[True,True])
    print(str(df["N_Variables"].sum()) + " variables")
    print(str(df["Niv_Gr_1"].len()) + " grupos")
    print(str(df["N_Subgrupos"].sum()) + " subgrupos")
    return df

def res_nivel_2(nivel_1):
    df = pl.DataFrame(df_all)
    variable_0 = "Niv_Gr_1"
    variable_1 = "Niv_Gr_2"
    variable_2 = "Niv_Gr_3"
    df = df.filter(
            pl.col("Niv_Gr_1") == nivel_1,
        ).group_by(variable_0,variable_1
        ).agg(
            pl.col(variable_0).len().alias("N_Variables"),
            pl.col(variable_2).n_unique().alias("N_Niv_Gr_2")
        ).sort(["N_Variables","N_Niv_Gr_2",],descending=[True,True])
    print(str(df["N_Variables"].sum()) + " variables")
    print(str(df["Niv_Gr_1"].len()) + " grupos")
    print(str(df["N_Niv_Gr_2"].sum()) + " subgrupos")
    return df

def res_nivel_3(nivel_1,nivel_2,variable_2):
    df = pl.DataFrame(df_all)
    variable_0 = "Niv_Gr_1"
    variable_1 = "Niv_Gr_2"
    df = df.filter(
            pl.col("Niv_Gr_1") == nivel_1,
            pl.col("Niv_Gr_2") == nivel_2
        ).group_by(
            [variable_0,variable_1,variable_2]
        ).agg(
            # pl.col("Niv_Gr_1").len().alias("N_Niv_Gr_2"),
            pl.col("Niv_Gr_2").len().alias("N_Variables"),
        ).sort("N_Variables",descending=True)
    print(str(len(df)) + " subgrupos")
    # print(str(df["N_Niv_Gr_2"].sum()) + " niveles")
    print(str(df["N_Variables"].sum()) + " variables")
    return df

def res_nivel_4(nivel_1,nivel_2,variable_2,def_variable_2,variable_3):
    df = pl.DataFrame(df_all)
    variable_0 = "Niv_Gr_1"
    variable_1 = "Niv_Gr_2"
    df = df.filter(
            pl.col("Niv_Gr_1") == nivel_1,
            pl.col("Niv_Gr_2") == nivel_2,
            pl.col(variable_2) == def_variable_2,
        ).group_by(
            [variable_0,variable_1,variable_2,variable_3]
        ).agg(
            pl.col(variable_3).len().alias("N_Variables"),
        ).sort("N_Variables",descending=True)
    return df

def res_nivel_5(
    nivel_1,nivel_2,
    variable_2,def_variable_2,
    variable_3,variable_4):
    df = pl.DataFrame(df_all)
    variable_0 = "Niv_Gr_1"
    variable_1 = "Niv_Gr_2"
    df = df.filter(
            pl.col("Niv_Gr_1") == nivel_1,
            pl.col("Niv_Gr_2") == nivel_2,
            pl.col(variable_2) == def_variable_2,
            # pl.col(variable_3) == def_variable_3,
        ).group_by(
            [variable_0,variable_1,variable_2,variable_3,variable_4]
        ).agg(
            pl.col(variable_4).len().alias("N_Variables"),
        ).sort("N_Variables",descending=True)
    return df

def res_nivel_6(
    nivel_1,nivel_2,
    variable_2,def_variable_2,
    variable_3,variable_4,variable_5):
    df = pl.DataFrame(df_all)
    variable_0 = "Niv_Gr_1"
    variable_1 = "Niv_Gr_2"
    df = df.filter(
            pl.col("Niv_Gr_1") == nivel_1,
            pl.col("Niv_Gr_2") == nivel_2,
            pl.col(variable_2) == def_variable_2,
            # pl.col(variable_3) == variable_3,
            # pl.col(variable_3) == def_variable_3,
        ).group_by(
            [variable_0,variable_1,variable_2,variable_3,variable_4,variable_5]
        ).agg(
            pl.col(variable_5).len().alias("N_Variables"),
        ).sort("N_Variables",descending=True)
    return df

def res_nivel_7(
    nivel_1,nivel_2,
    variable_2,def_variable_2,
    variable_3,variable_4,variable_5,variable_6):
    df = pl.DataFrame(df_all)
    variable_0 = "Niv_Gr_1"
    variable_1 = "Niv_Gr_2"
    df = df.filter(
            pl.col("Niv_Gr_1") == nivel_1,
            pl.col("Niv_Gr_2") == nivel_2,
            pl.col(variable_2) == def_variable_2,
            # pl.col(variable_3) == variable_3,
            # pl.col(variable_3) == def_variable_3,
        ).group_by(
            [variable_0,variable_1,variable_2,variable_3,variable_4,variable_5,variable_6]
        ).agg(
            pl.col(variable_5).len().alias("N_Variables"),
        ).sort("N_Variables",descending=True)
    return df

In [ ]:
# clave_asignada = "ff34cff7b0024ea39eb565fccb9f03b6"
# url = "https://bchapi-am.azure-api.net/api/v1/indicadores?formato=Json"
# hdr ={'Cache-Control': 'no-cache',
#       'clave': clave_asignada,}
# dfmeta = get_info(url, hdr)
# dfmeta

# Acceso a los Datos

Una vez que se crea el registro de usuario (usar el botón "Suscribirse" en el [sitio web](https://bchapi-am.developer.azure-api.net/), puede verse el procedimiento de registro e inicio de sesión en [YouTube](https://www.youtube.com/watch?v=8ZBllMSsKw4)), se necesita obtener una clave (ver explicación en [YouTube](https://www.youtube.com/watch?v=mV90s74OCfc)), ejecutando una consulta al catálogo de indicadores. Favor sustituir en el código la clave actual por su clave asignada.

Para conocer los datos disponibles, se debe ejecutar una consulta al catálogo de indicadores, que contiene información sobre las series disponibles:

* Nombre (código)
* Descripción
* Periodicidad
* Grupo
* Correlativo del Grupo

Como ejemplo de la consulta que contiene todas las variables, se muestran los primeros cinco elementos:


In [ ]:
clave_asignada = "ff34cff7b0024ea39eb565fccb9f03b6" #Favor ingresar la clave proporcionada.
url = "https://bchapi-am.azure-api.net/api/v1/indicadores?formato=Json"
hdr ={'Cache-Control': 'no-cache',
      'clave': clave_asignada,}
dfmeta = get_info(url, hdr)
dfmeta.head()

# Descripción de los Datos

## Grupos

Las variables están categorizadas en 8 grupos principales y 54 subgrupos. La nomenclatura utilizada permite dividir estos en un máximo de cinco niveles, tomando en cuenta el separador (-). Se explicarán los mismos a continuacion, ordenando de acuerdo al número de niveles (número de subgrupos).


In [ ]:
# Agregar niveles de Descripción y Grupo
df_all = dfmeta
df_all[['Niv_Descr_1','Niv_Descr_2','Niv_Descr_3','Niv_Descr_4','Niv_Descr_5','Niv_Descr_6','Niv_Descr_7','Niv_Descr_8']] = df_all['Descripcion'].str.split(
    '-',expand=True)
df_all["Niv_Descr"] = df_all.notnull().sum(axis=1) - 6
df_all[
    ['Niv_Gr_1','Niv_Gr_2','Niv_Gr_3','Niv_Gr_4','Niv_Gr_5']]  = df_all['Grupo'].str.split('-',expand=True)
df_all["Niv_Gr"] = df_all.notnull().sum(axis=1) - 2
df_all = pl.DataFrame(df_all)
df_all.write_csv(
        "api/variables.csv",
        separator=";")

# Resumen por Grupo
variable = "Grupo"
res_grupo = df_all.group_by(variable
    ).agg(
    pl.col("Id").len().alias("N_Variables"),
    ).sort("N_Variables",descending=True)
print(str(res_grupo["N_Variables"].sum()) + " variables")
print(str(len(res_grupo)) + " grupos")
res_grupo = pd.DataFrame(res_grupo)
res_grupo.columns = ["Grupo","N_Variables"]
res_grupo[
    ['Niv_Gr_1','Niv_Gr_2','Niv_Gr_3','Niv_Gr_4','Niv_Gr_5']]  = res_grupo['Grupo'].str.split('-',expand=True)
res_grupo["Niv_Gr"] = res_grupo.notnull().sum(axis=1) - 2
res_grupo = res_grupo.sort_values(
    ["Niv_Gr","N_Variables"], 
    ascending=[False,False])
res_grupo = pl.DataFrame(res_grupo)
res_grupo.write_csv(
        "api/grupos.csv",
        separator=";")

res_grupo

Los 8 grupos mencionados permiten consolidar los archivos de acuerdo con el origen de los datos en la página web del BCH, que en su mayoría pueden consultarse en los [reportes dinámicos](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos).

1. EOM = [Estadísticas de Operaciones Monetarias](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/operaciones-de-mercado-abierto)
2. ESR = [Estadísticas del Sector Real](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/sector-real)
3. ESE = [Estadísticas del Sector Externo](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/sector-externo)
4. EMF = [Estadísiticas Monetarias y Financieras (Sector Fiscal)](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/sector-fiscal)
5. EM = [Estadísticas Monetarias (TPM y RIN)](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/sector-monetario)
6. ESP = [Estadísticas de Sistema de Pagos](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/sistema-de-pagos)
7. EC = [Estadísticas de Tipo de Cambio](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/tipo-de-cambio)
8. EP = [Estadísticas de Precios](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/precios)

Los grupos se subdividen en 25 subgrupos:


In [ ]:
res = res_nivel_1()
res

A continuación se presentan los detalles de los grupos por nivel de agregación, ordenados de acuerdo al número de variables que contienen.

### EOM = Estadísticas de Operaciones Monetarias

Los datos están disponibles en la [página web del BCH](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/operaciones-de-mercado-abierto).


In [ ]:
res_nivel_2("EOM")

Contiene solamente un subgrupo:

- OMA = Operaciones de Mercado Abierto

#### OMA = Operaciones de Mercado Abierto

Cuatro niveles de agregación:

* 01 = Tasas de Rendimiento Diario de Valores Gubernamentales
* 02 = Créditos (Niv_Descr_4 = 1-3) e Inversiones (Niv_Descr_4 = 4-6): Monto, Número de Solicitudes y Tasa
* 03 = Monto, Número de Operaciones y Tasa
* 04 = Monto, Número de Operaciones y Tasa (Dólares - Lempiras)

En cada una de las tablas subsiguientes, la columna N_Variables corresponde al número de variables que contiene cada nivel.


In [ ]:
res_nivel_3("EOM","OMA","Niv_Gr_3").sort("Niv_Gr_3")

###### 01 = Tasas de Rendimiento Diario de Valores Gubernamentales 

Este es el nivel que contiene más variables, todas corresponden a las "Tasas de Rendimiento Diario de Valores Gubernamentales":


In [ ]:
res_nivel_4(
    "EOM","OMA",
    "Niv_Gr_3","01",
    "Niv_Descr_1")

Para listar todas las variables:


In [ ]:
res_nivel_6(
    "EOM","OMA",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_1").sort("Nombre")

La consulta para una serie individual puede realizarse solamente mediante el uso de la "Id" correspondiente; por ejemplo, el Id 701 da como resultado:


In [ ]:
Id = 1699
df = get_df(Id)
# get_plot(Id)
df

###### 02 = Créditos (Niv_Descr_4 = 1-3) e Inversiones (Niv_Descr_4 = 4-6): Monto, Número de Solicitudes y Tasa


In [ ]:
res_nivel_4(
    "EOM","OMA",
    "Niv_Gr_3","02",
    "Niv_Descr_5")

Todas las variables:


In [ ]:
res_nivel_7(
    "EOM","OMA",
    "Niv_Gr_3","02",
    "Id","Nombre","Niv_Descr_5","Niv_Descr_6").sort("Nombre")

Resultados de una sola variable:


In [ ]:
Id = 6261
df = get_df(Id)
# get_plot(Id)
df

###### 03 = Monto, Número de Operaciones y Tasa


In [ ]:
res_nivel_4(
    "EOM","OMA",
    "Niv_Gr_3","03",
    "Niv_Descr_5")

Todas las variables:


In [ ]:
res_nivel_6(
    "EOM","OMA",
    "Niv_Gr_3","03",
    "Id","Nombre","Niv_Descr_5").sort("Nombre")

Resultados de una sola variable:


In [ ]:
Id = 6267
df = get_df(Id)
# get_plot(Id)
df

###### 04 = Monto, Número de Operaciones y Tasa (Dólares - Lempiras)


In [ ]:
res_nivel_4(
    "EOM","OMA",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_5")

In [ ]:
res_nivel_5(
    "EOM","OMA",
    "Niv_Gr_3","04",
    "Niv_Descr_5","Niv_Descr_6")

Todas las variables:


In [ ]:
res_nivel_7(
    "EOM","OMA",
    "Niv_Gr_3","04",
    "Id","Nombre","Niv_Descr_5","Niv_Descr_6").sort("Nombre")

Resultados de una sola variable:


In [ ]:
Id = 6270
df = get_df(Id)
# get_plot(Id)
df

### ESR = Estadísticas del Sector Real


In [ ]:
res_nivel_2("ESR")

* COU = Cuadro de Oferta y Utilización
* CCI = Clasificación Cruzada Industria
* VAB = Valor Agregado Bruto
* PIBA = PIB Anual
* PIBT = PIB Trimestral
* ODA = Oferta y Demanda Agregada
* IMAE = Índice Mensual de la Actividad Económica

#### COU = Cuadro de Oferta y Utilización


In [ ]:
res_nivel_3("ESR","COU","Niv_Gr_3")

Detalla la interacción del producto con la actividad (el detalle se toma de acuerdo con la estructura de la columna "Nombre"). Para conocer la estructura de las estadísticas derivadas de este cuadro, consultar en la [página web del BCH](https://www.bch.hn/estadisticas-y-publicaciones-economicas/sector-real/cuentas-nacionales-anuales-base-2000/cuadro-oferta-y-utilizacion).

El orden está de acuerdo con el Nivel de Descripción 2 (Niv_Descr_2) y Nivel de Descripción 3 (Niv_Descr_3) de la columna Nombre, para los casos de:

* VALOR BRUTO DE PRODUCCIÓN
* CONSUMO INTERMEDIO
* VALOR AGREGADO
* OFERTA
* UTILIZACIÓN

Cada uno de estos detalles corresponde a una hoja de Excel en el archivo anual correspondiente.


In [ ]:
res_nivel_4(
    "ESR","COU",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1")

##### VALOR BRUTO DE PRODUCCIÓN

En el cuadro original, las filas son los productos a un nivel de dos dígitos de la Nomenclatura de Productos de Honduras (NPH) y las columnas son las actiivades económicas tomando en cuenta la Nomenclatura de Actividades Económicas de Honduras (NAEH).


In [ ]:
res_nivel_7(
    "ESR","COU",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1","Id","Nombre","Descripcion").filter(pl.col("Niv_Descr_1") == "VALOR BRUTO DE PRODUCCIÓN")

Resultados de una sola variable:


In [ ]:
Id = 10410
df = get_df(Id)
# get_plot(Id)
df

##### CONSUMO INTERMEDIO

En el cuadro original, las filas son los productos a un nivel de dos dígitos de la Nomenclatura de Productos de Honduras (NPH) y las columnas son las actiivades económicas tomando en cuenta la Nomenclatura de Actividades Económicas de Honduras (NAEH).


In [ ]:
res_nivel_7(
    "ESR","COU",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1","Id","Nombre","Descripcion").filter(pl.col("Niv_Descr_1") == "CONSUMO INTERMEDIO")

Resultados de una sola variable:


In [ ]:
Id = 7835
df = get_df(Id)
# get_plot(Id)
df

##### VALOR AGREGADO

En el cuadro original, las filas son los elementos indispensables para calcular el Valor Agregado Bruto, VAB (Remuneración de los Asalariados, Impuestos sobre la Producción y las Importaciones, Subvenciones, Excedente de Explotación, Ingreso Mixto y Consumo de Capital Fijo) y las columnas son las actiivades económicas tomando en cuenta la Nomenclatura de Actividades Económicas de Honduras (NAEH).


In [ ]:
res_nivel_7(
    "ESR","COU",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1","Id","Nombre","Descripcion").filter(pl.col("Niv_Descr_1") == "VALOR AGREGADO")

Resultados de una sola variable:


In [ ]:
Id = 10024
df = get_df(Id)
# get_plot(Id)
df

##### OFERTA

En el cuadro original, las filas son los productos a un nivel de dos dígitos de la Nomenclatura de Productos de Honduras (NPH) y las columnas son los componentes de la Oferta Agregada a Precios de Comprador (Producción, Importaciones CIF, Ajuste CIF/FOB, Impuestos y Subvenciones a los Productos y Márgenes de Distribución).


In [ ]:
res_nivel_7(
    "ESR","COU",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1","Id","Nombre","Descripcion").filter(pl.col("Niv_Descr_1") == "OFERTA")

Resultados de una sola variable:


In [ ]:
Id = 8878
df = get_df(Id)
# get_plot(Id)
df

##### UTILIZACIÓN

En el cuadro original, las filas son los productos a un nivel de dos dígitos de la Nomenclatura de Productos de Honduras (NPH) y las columnas son los componentes de la Utilización Agregada a Precios de Comprador (Consumo Intermedio, Exportciones FOB, Gasto de Consumo Final y Formación Bruta de Capital).


In [ ]:
res_nivel_7(
    "ESR","COU",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1","Id","Nombre","Descripcion").filter(pl.col("Niv_Descr_1") == "UTILIZACIÓN")

Tomar en cuenta que, para realizar el cálculo del VAB por NAEH, se debe sumar las columnas de Valor Bruto de Producción y restar las de Consumo Intermedio; adicionalmente, como los datos correspondientes a Impuestos sobre los Productos y los Márgenes de Distribución solamente se encuentran a nivel de NPH, no se puede tener un VAB total que se obtenga por suma de VAB a nivel de actividad individual, en vista que debe restarse las de Subvenciones a los productos, mismas que se encuentran en el Cuadro de Oferta solamente a nivel de NPH. Esta forma de obtener el VAB se conoce como el cálculo del PIB por el enfoque del Ingreso.

Como la Oferta y Utilización están detalladas a nivel de NPH, el VAB por NPH a partir de los dos detalles previos, se puede obtener por dos métodos:

1. Suma de Producción + Impuestos - Subvenciones a los Productos + Márgenes de Distribución (Oferta) - Consumo Intermedio (Utilización) => PIB por el enfoque de la Produccion.
2. Exportaciones FOB + Gasto de Consumo Final + Formación Bruta de Capital (Utilización) - Importaciones CIF - Ajuste CIF/FOB (Oferta) => PIB por el enfoque del Gasto.

Para saber el contenido de todas las variables:


In [ ]:
res_nivel_7(
    "ESR","COU",
    "Niv_Gr_3",
    "01",
    "Id","Niv_Descr_1","Nombre","Descripcion").sort("Id")

Resultados de una sola variable:


In [ ]:
Id = 9465
df = get_df(Id)
# get_plot(Id)
df

#### CCI = Clasificación Cruzada Industria Sector Institucional

Para conocer la estructura de las estadísticas derivadas de este cuadro, consultar en la [página web del BCH](https://www.bch.hn/estadisticas-y-publicaciones-economicas/sector-real/cuentas-nacionales-anuales-base-2000/clasificacion-cruzada-industria-sector-institucional).

El archivo contiene 3 hojas, que pueden filtrarse de acuerdo con el Nivel de Descripción 1 (Niv_Descr_1) de la variable:

* Producción Bruta
* Consumo Intermedio
* Valor Agregado

Los cuadros en cada hoja contienen los valores a nivel de Industria en las filas y a nivel de Sector Institucional en las columnas.

Debido a que el VAB implica tomar en cuenta impuestos y subvenciones a nivel de NPH, no es posible obtener el cálculo de esta variable por suma a partir de los cuadros del CCI (ver explicación en el apartado del COU).


In [ ]:
res_nivel_3("ESR","CCI","Niv_Descr_1")

"Niv_Descr_2" es la Industria (filas).


In [ ]:
res_nivel_5(
    "ESR","CCI",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1","Niv_Descr_2")

"Niv_Descr_3" es el Sector Institucional (columnas).


In [ ]:
res_nivel_5(
    "ESR","CCI",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1","Niv_Descr_3")

Para saber el contenido de todas las variables:


In [ ]:
res_nivel_7(
    "ESR","CCI",
    "Niv_Gr_3",
    "01",
    "Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_3")

Resultados de una sola variable:


In [ ]:
Id = 7467
df = get_df(Id)
# get_plot(Id)
df

#### VAB = Valor Agregado Bruto


In [ ]:
res_nivel_3("ESR","VAB","Niv_Gr_3")

##### Valor y Variación, Corrientes y Constantes, por NAEH


In [ ]:
res_nivel_4(
    "ESR","VAB",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_4")

In [ ]:
res_nivel_5(
    "ESR","VAB",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_4","Niv_Descr_5")

Para saber el contenido de todas las variables:


In [ ]:
res_nivel_7(
    "ESR","VAB",
    "Niv_Gr_3",
    "01",
    "Id","Niv_Descr_4","Niv_Descr_5","Niv_Descr_6")

Resultados de una sola variable:


In [ ]:
Id = 220
df = get_df(Id)
# get_plot(Id)
df

##### Precio y Variación, Corrientes y Constantes, por NAEH


In [ ]:
res_nivel_4(
    "ESR","VAB",
    "Niv_Gr_3",
    "02",
    "Niv_Descr_4")

In [ ]:
res_nivel_5(
    "ESR","VAB",
    "Niv_Gr_3",
    "02",
    "Niv_Descr_4","Niv_Descr_5")

Notar que aquí el Nivel de Descripción 6 (Niv_Descr_6) no es una buena opción de desagregación, puesto que los componentes son "Actividad Económica" y "Productos Alimenticios, Bebidas y Tabaco"; esta última está detallada en mayor grado en el Nivel de Descripción 7 (Niv_Descr_7).

Para saber el contenido de todas las variables:


In [ ]:
res_nivel_7(
    "ESR","VAB",
    "Niv_Gr_3",
    "02",
    "Id","Niv_Descr_4","Niv_Descr_5","Niv_Descr_7")

Resultados de una sola variable:


In [ ]:
Id = 273
df = get_df(Id)
# get_plot(Id)
df

#### PIBA = PIB Anual

La información se encuentra en la viñeta "Sector Real" de la [página web del BCH](https://www.bch.hn/estadisticas-y-publicaciones-economicas/reportes-dinamicos/sector-real).


In [ ]:
res_nivel_3("ESR","PIBA","Niv_Descr_3")

##### 01 Valores Corrientes y Constantes, por Actividad Económica


In [ ]:
res_nivel_4(
    "ESR","PIBA",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_4")

* Rama de Actividad Económica (RAE)
* Series Agregadas
* Variación RAE
* Variación Series Agregadas


In [ ]:
res_nivel_6(
    "ESR","PIBA",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_4","Niv_Descr_5","Niv_Descr_6")

Para saber el contenido de todas las variables:


In [ ]:
res_nivel_7(
    "ESR","PIBA",
    "Niv_Gr_3",
    "01",
    "Id","Niv_Descr_4","Niv_Descr_5","Niv_Descr_6")

Resultados de una sola variable:


In [ ]:
Id = 101
df = get_df(Id)
# get_plot(Id)
df

##### 02 Precios y Variación, por Componente del Gasto


In [ ]:
res_nivel_4(
    "ESR","PIBA",
    "Niv_Gr_3",
    "02",
    "Niv_Descr_4")

Tipo de Valores ("Niv_Descr_5"):

* Corrientes
* Constantes


In [ ]:
res_nivel_6(
    "ESR","PIBA",
    "Niv_Gr_3",
    "02",
    "Niv_Descr_4","Niv_Descr_5","Niv_Descr_6")

Para saber el contenido de todas las variables:


In [ ]:
res_nivel_7(
    "ESR","PIBA",
    "Niv_Gr_3",
    "02",
    "Id","Niv_Descr_4","Niv_Descr_5","Niv_Descr_6")

Resultados de una sola variable:


In [ ]:
Id = 194
df = get_df(Id)
# get_plot(Id)
df

##### 03 Precios y Variación, por Componente del Ingreso


In [ ]:
res_nivel_4(
    "ESR","PIBA",
    "Niv_Gr_3",
    "03",
    "Niv_Descr_4")

* Remuneración de los asalariados
* Impuestos netos de subvenciones sobre la producción y las importaciones
* Excedente de Explotación bruto
* Ingreso Mixto bruto
* Producto Interno Bruto (pm)


In [ ]:
res_nivel_5(
    "ESR","PIBA",
    "Niv_Gr_3",
    "03",
    "Niv_Descr_4","Niv_Descr_5")

Para saber el contenido de todas las variables:


In [ ]:
res_nivel_7(
    "ESR","PIBA",
    "Niv_Gr_3",
    "03",
    "Id","Nombre","Niv_Descr_4","Niv_Descr_5")

Resultados de una sola variable:


In [ ]:
Id = 202
df = get_df(Id)
# get_plot(Id)
df

##### 04 Dólares y Lempiras


In [ ]:
res_nivel_4(
    "ESR","PIBA",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_4")

* Ingreso Nacional
* Producto Interno Bruto
* Producto Nacional Bruto


In [ ]:
res_nivel_5(
    "ESR","PIBA",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_4","Niv_Descr_6")

Para saber el contenido de todas las variables:


In [ ]:
res_nivel_7(
    "ESR","PIBA",
    "Niv_Gr_3",
    "04",
    "Id","Nombre","Niv_Descr_4","Niv_Descr_6")

Resultados de una sola variable:


In [ ]:
Id = 211
df = get_df(Id)
# get_plot(Id)
df

#### PIBT = PIB Trimestral

Esta información se obtiene a partir de los datos del [Portal de Consulta del Sistema de Cuentas Trimestrales](https://sisee.bch.hn/SCNT/PortalPrincipal.aspx).


In [ ]:
res_nivel_3("ESR","PIBT","Niv_Gr_3")

* PROD = PIB Trimestral por el Enfoque de la Producción
* GAST = PIB Trimestral por el Enfoque del Gasto
* TDC = Tasas de Crecimiento

##### PROD = PIB Trimestral por el Enfoque de la Producción

* Serie Original
* Serie Desestacionalizada
* Valores Corrientes
* Valores Constantes

Para saber el contenido de todas las variables:


In [ ]:
res_nivel_7(
    "ESR","PIBT",
    "Niv_Gr_3",
    "PROD",
    "Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_4")

Resultados de una sola variable:


In [ ]:
Id = 2
df = get_df(Id)
# get_plot(Id)
df

##### GAST = PIB Trimestral por el Enfoque del Gasto

* Serie Original
* Serie Desestacionalizada
* Valores Corrientes
* Valores Constantes

Para saber el contenido de todas las variables:


In [ ]:
res_nivel_7(
    "ESR","PIBT",
    "Niv_Gr_3",
    "GAST",
    "Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_4")

Resultados de una sola variable:


In [ ]:
Id = 8
df = get_df(Id)
# get_plot(Id)
df

##### TDC = Tasas de Crecimiento

* Serie Original
* Serie Desestacionalizada
* Valores Corrientes
* Valores Constantes

Para saber el contenido de todas las variables:


In [ ]:
res_nivel_6(
    "ESR","PIBT",
    "Niv_Gr_3",
    "TDC",
    "Id","Nombre","Niv_Descr_1")

Resultados de una sola variable:


In [ ]:
Id = 98
df = get_df(Id)
# get_plot(Id)
df

#### ODA = Oferta y Demanda Agregada

* Oferta Global
* Demanda Global


In [ ]:
res_nivel_3("ESR","ODA","Niv_Descr_1")

##### Oferta Global

Estas tablas están agrupadas de dos maneras:

1. Variables con valores en "Niv_Descr_2" y "Niv_Descr_3", corresponde a las siguientes series:

* AGRICULTURA, GANADERÍA, CAZA, SILVICULTURA Y PESCA (A PRECIOS BÁSICOS)
* COMERCIO, HOTELES Y RESTAURANTES (A PRECIOS BÁSICOS)
* CONSTRUCCIÓN (A PRECIOS BÁSICOS)
* EXPLOTACIÓN DE MINAS Y CANTERAS (A PRECIOS BÁSICOS)
* INDUSTRIA MANUFACTURERA (A PRECIOS BÁSICOS)
* MENOS: SERVICIOS DE INTERMEDIACIÓN FINANCIERA MEDIDOS INDIRECTAMENTE (A PRECIOS BÁSICOS)

El tipo de valor ("Niv_Descr_4") de dichas variables es:

* EN MILLONES DE LEMPIRAS CONSTANTES
* EN MILLONES DE LEMPIRAS CORRIENTES
* VARIACIONES RELATIVAS CONSTANTE
* VARIACIONES RELATIVAS CORRIENTES


In [ ]:
res_nivel_7(
    "ESR","ODA",
    "Niv_Descr_1",
    "OFERTA GLOBAL",
    "Id","Niv_Descr_2","Niv_Descr_3","Niv_Descr_4"
    ).filter((pl.col('Niv_Descr_2') != ""))

Resultados de una sola variable:


In [ ]:
Id = 11518
df = get_df(Id)
# get_plot(Id)
df

2. Variables sin valores en "Niv_Descr_2" y "Niv_Descr_3", corresponde a las siguientes series:

* IMPORTACIONES DE BIENES Y SERVICIOS C.I.F.
* IMPUESTOS NETOS DE SUBVENCIONES SOBRE LA PRODUCCIÓN Y LAS IMPORTACIONES
* OFERTA GLOBAL
* OTROS SERVICIOS (A PRECIOS BÃSICOS)
* PRODUCTO INTERNO BRUTO A PRECIOS DE MERCADO
* TOTAL VALOR AGREGADO BRUTO A PRECIOS BÁSICOS
* VALOR AGREGADO BRUTO
* VALOR AGREGADO BRUTO A PRECIOS BÁSICOS

El tipo de valor ("Niv_Descr_5") de dichas variables es:

* EN MILLONES DE LEMPIRAS CONSTANTES
* EN MILLONES DE LEMPIRAS CORRIENTES
* VARIACIONES RELATIVAS CONSTANTE
* VARIACIONES RELATIVAS CORRIENTES

Para saber el contenido de todas las variables:


In [ ]:
res_nivel_7(
    "ESR","ODA",
    "Niv_Descr_1",
    "OFERTA GLOBAL",
    "Id","Niv_Descr_2","Niv_Descr_4","Niv_Descr_5"
    ).filter((pl.col('Niv_Descr_2') == ""))

Resultados de una sola variable:


In [ ]:
Id = 11473
df = get_df(Id)
# get_plot(Id)
df

##### Demanda Global

Estas tablas están agrupadas de dos maneras:

1. Variables con valores en "Niv_Descr_2" y "Niv_Descr_3", corresponde a las siguientes series:

* FORMACIÓN BRUTA DE CAPITAL FIJO
* GASTOS DE CONSUMO FINAL

De "Niv_Descr_3" se obtiene:

* SECTOR PRIVADO
* SECTOR PÚBLICO

El tipo de valor ("Niv_Descr_4") de dichas variables es:

* EN MILLONES DE LEMPIRAS CONSTANTES
* EN MILLONES DE LEMPIRAS CORRIENTES
* VARIACIONES RELATIVAS CONSTANTE
* VARIACIONES RELATIVAS CORRIENTES


In [ ]:
res_nivel_7(
    "ESR","ODA",
    "Niv_Descr_1",
    "DEMANDA GLOBAL",
    "Id","Niv_Descr_2","Niv_Descr_3","Niv_Descr_4"
    ).filter((pl.col('Niv_Descr_2') != ""))

Resultados de una sola variable:


In [ ]:
Id = 11457
df = get_df(Id)
# get_plot(Id)
df

2. Variables sin valores en "Niv_Descr_2" y "Niv_Descr_3", corresponde a las siguientes series:

* DEMANDA EXTERNA
* DEMANDA GLOBAL
* DEMANDA INTERNA
* EXPORTACIÓN DE BIENES Y SERVICIOS F.O.B.
* FORMACIÓN BRUTA DE CAPITAL FIJO
* GASTOS DE CONSUMO FINAL
* INVERSIÓN INTERNA BRUTA
* VARIACIÓN DE EXISTENCIAS

El tipo de valor de dichas variables es:

* EN MILLONES DE LEMPIRAS CONSTANTES
* EN MILLONES DE LEMPIRAS CORRIENTES
* VARIACIONES RELATIVAS CONSTANTE
* VARIACIONES RELATIVAS CORRIENTES

Para saber el contenido de todas las variables:


In [ ]:
res_nivel_7(
    "ESR","ODA",
    "Niv_Descr_1",
    "DEMANDA GLOBAL",
    "Id","Niv_Descr_2","Niv_Descr_4","Niv_Descr_5"
    ).filter((pl.col('Niv_Descr_2') == ""))

Resultados de una sola variable:


In [ ]:
Id = 11440
df = get_df(Id)
# get_plot(Id)
df

#### IMAE = Índice Mensual de la Actividad Económica

Las actividades ("Niv_Descr_4") son:

* Agricultura, Ganaderia, Silvicultura y Pesca 
* Minas y Canteras 
*  Industria Manufacturera 
* Electricidad y Agua 
* ConstrucciÃ³n 
* Comercio 
* Hoteles y Restaurantes 
* Transporte y Almacenamiento 
* Correo y Telecomunicaciones 
* Inter. Financiera, Seguros y Fondos de Pensiones 
* Otros Servicios 
* Serie Original 
* Tendencia Ciclo 

El tipo de valor ("Niv_Descr_5") de dichas variables es:

* Acumulada
* Interanual
* Índice


In [ ]:
res_nivel_3("ESR","IMAE","Niv_Descr_2")

Para saber el contenido de todas las variables:


In [ ]:
res_nivel_6(
    "ESR","IMAE",
    "Niv_Descr_2",
    "IMAE",
    "Id","Niv_Descr_4","Niv_Descr_5")

Resultados de una sola variable:


In [ ]:
Id = 372
df = get_df(Id)
# get_plot(Id)
df

### ESE = Estadísticas del Sector Externo


In [ ]:
res_nivel_2("ESE")

* DE = Deuda Externa
* CEB = Comercio Exterior de Bienes
* BP = Balanza de Pagos
* IES = Ingresos y Egresos de Servicios
* PII = Posición de Inversión Inernacional
* IED = Inversión Extranjera Directa

#### DE = Deuda Externa


In [ ]:
res_nivel_3("ESE","DE","Niv_Gr_3")

##### 01 - Deudor

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "ESE","DE",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_1","Niv_Descr_2").sort("Nombre")

Resultados de una sola variable:


In [ ]:
Id = 6710
df = get_df(Id)
# get_plot(Id)
df

##### 02 - Acreedor

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "ESE","DE",
    "Niv_Gr_3","02",
    "Id","Nombre","Niv_Descr_1","Niv_Descr_2").sort("Nombre")

Resultados de una sola variable:


In [ ]:
Id = 6714
df = get_df(Id)
# get_plot(Id)
df

##### 03 - Por Institución

* Banco Central
* Gobierno general
* Otras sociedades financieras
* Otros Sectores
* Saldo bruto de la deuda externa
* Sociedades no financieras


In [ ]:
res_nivel_4(
    "ESE","DE",
    "Niv_Gr_3",
    "03",
    "Niv_Descr_1")

* Asignación de Derechos especiales de giro
* Asignación de Derechos especiales de giro 
* Créditos y anticipos comerciales
* Créditos y anticipos comerciales 
* Moneda y depósitos
* Moneda y depósitos 
* Otros pasivos de deuda
* Otros pasivos de deuda 
* Préstamos
* Préstamos 
* Tí­tulos de deuda
* Tí­tulos de deuda 
* Saldo bruto de la deuda externa
* Saldo bruto de la deuda externa 


In [ ]:
res_nivel_4(
    "ESE","DE",
    "Niv_Gr_3",
    "03",
    "Niv_Descr_2")

In [ ]:
res_nivel_7(
    "ESE","DE",
    "Niv_Gr_3","03",
    "Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_3")

Para ver todas las variables (Nota: el nivel de descripción sigue):


In [ ]:
df_all.filter(
    pl.col("Niv_Gr_1") == "ESE",
    pl.col("Niv_Gr_2") == "DE",
    pl.col("Niv_Gr_3") == "03"
    ).select(pl.all().exclude(
        "Niv_Descr_6","Niv_Descr_7","Niv_Descr_8",
        "Niv_Gr_4","Niv_Gr_5"))

Resultados de una sola variable:


In [ ]:
Id = 6720
df = get_df(Id)
# get_plot(Id)
df

##### 04 - Pagos que vencen en un año o menos

* Gobierno general
* Banco Central
* Otros Sectores
* Otras sociedades financieras
* Sociedades no financieras
* Deuda externa total a corto plazo


In [ ]:
res_nivel_4(
    "ESE","DE",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_1")

* A corto plazo según el vencimiento original
* A largo plazo exigible dentro de un año o antes
* Deuda externa total a corto plazo


In [ ]:
res_nivel_4(
    "ESE","DE",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_2")

* Asignación de Derechos especiales de giro
* Moneda y depósitos
* Tí­tulos de deuda
* Préstamos
* Créditos y anticipos comerciales
* Otros pasivos de deuda
* Deuda externa total a corto plazo (según el vencimiento residual)


In [ ]:
res_nivel_4(
    "ESE","DE",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_3")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "ESE","DE",
    "Niv_Gr_3",
    "04",
    "Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_3").sort("Id")

**OJO: REVISAR LA CONSULTA A PARTIR DE 7392 HASTA 7446**

Resultados de una sola variable:


In [ ]:
Id = 7447 # 7392 - 7446
df = get_df(Id)
# get_plot(Id)
df

#### CEB = Comercio Exterior de Bienes


In [ ]:
res_nivel_3("ESE","CEB","Niv_Gr_3")

##### 01 - Productos

* Banano
* Café
* Aceite de Palma
* Camarones
* Melones y Sandí­as
* Legumbres y Hortalizas
* Puros o Cigarros
* Textiles
* Jabones
* Plásticos y su Manufacturas
* Papel y Cartón


In [ ]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_5")

* Volumen
* Valor
* Precio


In [ ]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_6")

* Trimestral
* Anual


In [ ]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_7")

**OJO: "Niv_Descr_7" y "Niv_Descr_8" son iguales**

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "ESE","CEB",
    "Niv_Gr_3",
    "01","Id","Niv_Descr_5","Niv_Descr_6","Niv_Descr_7")

Resultados de una sola variable:


In [ ]:
Id = 441
df = get_df(Id)
# get_plot(Id)
df

##### 02 - Combustibles

* Gasolina regular
* Bunker
* Diesel
* Total combustibles
* Gasolina superior
* Kerosene
* LPG


In [ ]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "02",
    "Niv_Descr_5")

* Barriles
* Valor
* Precio


In [ ]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "02",
    "Niv_Descr_6")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "ESE","CEB",
    "Niv_Gr_3",
    "02","Id","Nombre","Niv_Descr_5","Niv_Descr_6")

Resultados de una sola variable:


In [ ]:
Id = 503
df = get_df(Id)
get_plot(Id)
df

##### 03

* Importaciones AE 
* Importaciones BT Sección 
* Importaciones Secciones 


In [ ]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "03",
    "Niv_Descr_1")

Los dos primeros grupos solamente tienen un nivel de descripción:

###### Importaciones AE


In [ ]:
res_nivel_7(
    "ESE","CEB",
    "Niv_Gr_3",
    "03","Id","Nombre","Niv_Descr_1","Niv_Descr_2"
    ).filter(
    pl.col("Niv_Descr_1") == "Importaciones AE ")

###### Importaciones BT Sección 


In [ ]:
res_nivel_7(
    "ESE","CEB",
    "Niv_Gr_3",
    "03","Id","Nombre","Niv_Descr_1","Niv_Descr_2"
    ).filter(
    pl.col("Niv_Descr_1") == "Importaciones BT Sección ")

El tercer grupo tiene dos niveles de descripción

###### Importaciones Secciones 

* Valores en Millones de USD
* Peso en Kilos


In [ ]:
res_nivel_7(
    "ESE","CEB",
    "Niv_Gr_3",
    "03","Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_3"
    ).filter(
    pl.col("Niv_Descr_1") == "Importaciones Secciones ")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "ESE","CEB",
    "Niv_Gr_3",
    "03","Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_3"
    )

Resultados de una sola variable:


In [ ]:
Id = 6517
df = get_df(Id)
# get_plot(Id)
df

##### 04 AE y BT

* Exportaciones AE
* Exportaciones BT


In [ ]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_1")

* Producto
* Grupo Producto


In [ ]:
res_nivel_4(
    "ESE","CEB",
    "Niv_Gr_3",
    "04",
    "Niv_Descr_2")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "ESE","CEB",
    "Niv_Gr_3",
    "04","Id","Niv_Descr_1","Niv_Descr_2","Niv_Descr_3"
    )

Resultados de una sola variable:


In [ ]:
Id = 6554
df = get_df(Id)
# get_plot(Id)
df

#### BP = Balanza de Pagos


In [ ]:
res_nivel_3("ESE","BP","Niv_Gr_3")

* Anual
* Mensual


In [ ]:
res_nivel_4(
    "ESE","BP",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_2")

Por Componentes:


In [ ]:
res_nivel_4(
    "ESE","BP",
    "Niv_Gr_3",
    "01",
    "Niv_Descr_1")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "ESE","BP",
    "Niv_Gr_3",
    "01","Id","Nombre","Niv_Descr_2","Niv_Descr_1").sort("Niv_Descr_2")

Resultados de una sola variable:


In [ ]:
Id = 6292
df = get_df(Id)
# get_plot(Id)
df

#### IES - Ingresos y Egresos de Servicios


In [ ]:
res_nivel_3("ESE","IES","Niv_Gr_3")

Periodicidad:

* Anual
* Trimestral


In [ ]:
res_nivel_4(
    "ESE","IES",
    "Niv_Descr_3",
    "01","Niv_Descr_5")

Ingresos y Egresos:

* Exportaciones (Ingresos)
* Importaciones (Egresos)


In [ ]:
res_nivel_4(
    "ESE","IES",
    "Niv_Descr_3",
    "01","Niv_Descr_6")

Variables:

* Transporte
* Viajes
* Servicios de Comunicaciones
* Otros Servicios Empresariales
* Servicios de Gobierno
* Otros Servicios
* Servicios de Seguro
* Servicios de InformÃ¡tica e Información
* Servicios de RegalÃ­as y Derechos de Licencia


In [ ]:
res_nivel_4(
    "ESE","IES",
    "Niv_Descr_3",
    "01","Niv_Descr_7")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "ESE","IES",
    "Niv_Gr_3",
    "01","Id","Niv_Descr_5","Niv_Descr_6","Niv_Descr_7")

Resultados de una sola variable:


In [ ]:
Id = 388
# df = get_df(Id)
get_plot(Id)
df

#### PII = Posición de Inversión Inernacional


In [ ]:
res_nivel_3("ESE","PII","Niv_Gr_3")

Agrupación:

* Posición de Inversión Internacional por Categorí­a  
* Posición de los Activos Financieros con no Residentes por Componente 
* Posición de los Pasivos Financieros con no Residentes por Componente 
* Posición de Inversión Internacional Neta por Componente 


In [ ]:
res_nivel_4(
    "ESE","PII",
    "Niv_Gr_3",
    "01","Niv_Descr_5")

Cuentas:

* Activos 
* Pasivos
* PII Neta
* Inversió³n extranjera directa
* Inversión de cartera
* Derivados financieros
* Otra inversión
* Activos de reservas lí­quidos oficiales


In [ ]:
res_nivel_4(
    "ESE","PII",
    "Niv_Gr_3",
    "01","Niv_Descr_6")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "ESE","PII",
    "Niv_Gr_3",
    "01","Id","Nombre","Niv_Descr_5","Niv_Descr_6")

Resultados de una sola variable:


In [ ]:
Id = 496
df = get_df(Id)
# get_plot(Id)
df

#### IED = Inversión Extranjera Directa


In [ ]:
res_nivel_3("ESE","IED","Niv_Gr_3")

Para ver todas las variables:


In [ ]:
res_nivel_6(
    "ESE","IED",
    "Niv_Gr_3",
    "01","Id","Nombre","Niv_Descr_4")

Resultados de una sola variable:


In [ ]:
Id = 409
df = get_df(Id)
# get_plot(Id)
df

### EMF = Estadísiticas Monetarias y Financieras


In [ ]:
res_nivel_2("EMF")

#### AMCC = Agregados Monetarios, Crédito y Captación


In [ ]:
res_nivel_3("EMF","AMCC","Niv_Gr_3").sort("Niv_Gr_3")

##### 01 Agregados Monetarios, Crédito y Captación

Grupos:


In [ ]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","01",
    "Niv_Descr_5")

Variables:


In [ ]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","01",
    "Niv_Descr_6")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EMF","AMCC",
    "Niv_Gr_3","01",
    "Id","Niv_Descr_5","Niv_Descr_6","Niv_Descr_4")

Resultados de una sola variable:


In [ ]:
Id = 548
# df = get_df(Id)
get_plot(Id)
df

##### 02 Captación: Bancos Comerciales y Otras Sociedades de Depósito (Variación y Saldo)


In [ ]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","02",
    "Niv_Descr_1")

In [ ]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","02",
    "Niv_Descr_3")

In [ ]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","02",
    "Niv_Descr_4")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EMF","AMCC",
    "Niv_Gr_3","02",
    "Id","Niv_Descr_1","Niv_Descr_3","Niv_Descr_4")

Resultados de una sola variable:


In [ ]:
Id = 622
df = get_df(Id)
# get_plot(Id)
df

##### 03 Crédito de los Bancos Comerciales


In [ ]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","03",
    "Niv_Descr_1")

In [ ]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","03",
    "Niv_Descr_4")

In [ ]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","03",
    "Niv_Descr_3")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EMF","AMCC",
    "Niv_Gr_3","03",
    "Id","Niv_Descr_1","Niv_Descr_3","Niv_Descr_4")

Resultados de una sola variable:


In [ ]:
Id = 638
df = get_df(Id)
# get_plot(Id)
df

##### 04 Crédito Otras Sociedades de Depósito


In [ ]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","04",
    "Niv_Descr_1")

In [ ]:
res_nivel_5(
    "EMF","AMCC",
    "Niv_Gr_3","04",
    "Niv_Descr_3","Niv_Descr_4")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EMF","AMCC",
    "Niv_Gr_3","04",
    "Id","Niv_Descr_1","Niv_Descr_3","Niv_Descr_4")

Resultados de una sola variable:


In [ ]:
Id = 648
df = get_df(Id)
# get_plot(Id)
df

##### 05 Saldo de Préstamos (Millones de Lempiras)


In [ ]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","05",
    "Niv_Descr_1")

* Variación
* Saldo

De los siguientes sectores:

* Agropecuaria
* Comercio
* Consumo
* Industria
* Propiedad Raí­z
* Servicios
* Total

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EMF","AMCC",
    "Niv_Gr_3","05",
    "Id","Niv_Descr_1","Niv_Descr_3","Niv_Descr_4").sort("Niv_Descr_4")

Resultados de una sola variable:


In [ ]:
Id = 650
df = get_df(Id)
# get_plot(Id)
df

##### 06 Préstamos Nuevos (Flujos acumulados en millones de lempiras)


In [ ]:
res_nivel_4(
    "EMF","AMCC",
    "Niv_Gr_3","06",
    "Niv_Descr_1")

* Variación
* Saldo

De los siguientes sectores:

* Agropecuaria
* Comercio
* Consumo
* Industria
* Propiedad Raí­z
* Servicios
* Total


In [ ]:
res_nivel_5(
    "EMF","AMCC",
    "Niv_Gr_3","06",
    "Niv_Descr_1","Niv_Descr_4")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EMF","AMCC",
    "Niv_Gr_3","06",
    "Id","Niv_Descr_1","Niv_Descr_3","Niv_Descr_4").sort("Niv_Descr_4")

Resultados de una sola variable:


In [ ]:
Id = 663
df = get_df(Id)
# get_plot(Id)
df

#### TI = Tasas de Interés


In [ ]:
res_nivel_3("EMF","TI","Niv_Gr_3").sort("Niv_Gr_3")

##### 01 Tasas de Interés, Activa y Pasiva, por Moneda

Por Tipo:

* Activa Préstamos 
* Pasiva Ahorros 
* Pasiva Cheques 
* Pasiva Plazo 


In [ ]:
res_nivel_4(
    "EMF","TI",
    "Niv_Gr_3","01",
    "Niv_Descr_5")

Por Moneda:

* Moneda Nacional
* Moneda Extranjera

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EMF","TI",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_5","Niv_Descr_6")

Resultados de una sola variable:


In [ ]:
Id = 587
df = get_df(Id)
# get_plot(Id)
df

##### 02 Por Actividad


In [ ]:
res_nivel_4(
    "EMF","TI",
    "Niv_Gr_3","02",
    "Niv_Descr_5")

Por Moneda:

* Moneda Nacional
* Moneda Extranjera


In [ ]:
res_nivel_5(
    "EMF","TI",
    "Niv_Gr_3","02",
    "Niv_Descr_1","Niv_Descr_4")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EMF","TI",
    "Niv_Gr_3","02",
    "Id","Nombre","Niv_Descr_5","Niv_Descr_4").sort("Niv_Descr_4")

Resultados de una sola variable:


In [ ]:
Id = 605
df = get_df(Id)
# get_plot(Id)
df

##### 03 Tasas de Interés Anuales

* Tasas de Interés Anuales Mí­nimas y Máximas del Sistema Bancario Comercial
* Tasas de Interés Anuales Promedio Ponderadas del Sistema Financiero


In [ ]:
res_nivel_4(
    "EMF","TI",
    "Niv_Gr_3","03",
    "Niv_Descr_6")

* Activa
* Pasiva

Por Unidad de Medida:

* Máxima
* Mínima
* Promedio

Por Moneda:

* Nacional
* Extranjera

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EMF","TI",
    "Niv_Gr_3","03",
    "Id","Nombre","Niv_Descr_6","Niv_Descr_5").sort("Niv_Descr_6")

Resultados de una sola variable:


In [ ]:
Id = 689
df = get_df(Id)
# get_plot(Id)
df

#### PFS = Panorama de las Sociedades Financieras


In [ ]:
res_nivel_3("EMF","PFS","Niv_Gr_3").sort("Niv_Gr_3")

##### 01 Bancos Comerciales, Activos y Pasivos Externos

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EMF","PFS",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_2","Niv_Descr_1").sort("Nombre")

Resultados de una sola variable:


In [ ]:
Id = 677
df = get_df(Id)
# get_plot(Id)
df

##### 02 Activos y Pasivos Externos


In [ ]:
res_nivel_4(
    "EMF","PFS",
    "Niv_Gr_3","02",
    "Niv_Descr_1")

In [ ]:
res_nivel_4(
    "EMF","PFS",
    "Niv_Gr_3","02",
    "Niv_Descr_2")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EMF","PFS",
    "Niv_Gr_3","02",
    "Id","Nombre","Niv_Descr_1","Niv_Descr_2")

Resultados de una sola variable:


In [ ]:
Id = 687
df = get_df(Id)
# get_plot(Id)
df

#### EBM = Emisión y Base Monetaria


In [ ]:
res_nivel_3("EMF","EBM","Niv_Gr_3").sort("Niv_Gr_3")

In [ ]:
res_nivel_4(
    "EMF","EBM",
    "Niv_Gr_3","01",
    "Niv_Descr_4")

In [ ]:
res_nivel_5(
    "EMF","EBM",
    "Niv_Gr_3","01",
    "Niv_Descr_4","Niv_Descr_5")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EMF","EBM",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_4","Niv_Descr_5")

Resultados de una sola variable:


In [ ]:
Id = 523
df = get_df(Id)
# get_plot(Id)
df

### ESP = Estadísticas de Sistema de Pagos


In [ ]:
res_nivel_2("ESP")

#### ACH = Operaciones de Crédito Compensadas en Ceproban y Liquidadas en BCH


In [ ]:
res_nivel_3("ESP","ACH","Niv_Gr_3").sort("Niv_Gr_3")

In [ ]:
res_nivel_4(
    "ESP","ACH",
    "Niv_Gr_3","01",
    "Niv_Descr_5")

In [ ]:
res_nivel_4(
    "ESP","ACH",
    "Niv_Gr_3","01",
    "Niv_Descr_6")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "ESP","ACH",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_5","Niv_Descr_6").sort("Nombre")

Resultados de una sola variable:


In [ ]:
Id = 611
df = get_df(Id)
# get_plot(Id)
df

#### CCECH = Cheques Compensados en Ceproban y Liquidados en BCH


In [ ]:
res_nivel_3("ESP","CCECH","Niv_Gr_3")

In [ ]:
res_nivel_4(
    "ESP","CCECH",
    "Niv_Gr_3","01",
    "Niv_Descr_5")

In [ ]:
res_nivel_4(
    "ESP","CCECH",
    "Niv_Gr_3","01",
    "Niv_Descr_6")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "ESP","CCECH",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_5","Niv_Descr_6")

Resultados de una sola variable:


In [ ]:
Id = 615
df = get_df(Id)
# get_plot(Id)
df

### EM = Estadísticas Monetarias (TPM y RIN)


In [ ]:
res_nivel_2("EM")

#### RIN = Reservas Internacionales Netas


In [ ]:
res_nivel_3("EM","RIN","Niv_Gr_3").sort("Niv_Gr_3")

* Saldo
* Variación

Variables:

* Reservas Internacionales Netas
* Activos de Reserva Oficial


In [ ]:
res_nivel_4(
    "EM","RIN",
    "Niv_Gr_3","01",
    "Niv_Descr_4")

In [ ]:
res_nivel_4(
    "EM","RIN",
    "Niv_Gr_3","01",
    "Niv_Descr_5")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EM","RIN",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_4","Niv_Descr_5")

Resultados de una sola variable:


In [ ]:
Id = 532
df = get_df(Id)
# get_plot(Id)
df

#### TPM = Tasa de Política Monetaria


In [ ]:
res_nivel_3("EM","TPM","Niv_Gr_3")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EM","TPM",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_1","Niv_Descr_2").sort("Niv_Descr_2")

Resultados de una sola variable:


In [ ]:
Id = 699
df = get_df(Id)
# get_plot(Id)
df

### EC = Estadísticas Cambiarias


In [ ]:
res_nivel_2("EC")

#### TCR = Tipo de Cambio de Referencia


In [ ]:
res_nivel_3("EC","TCR","Niv_Gr_3").sort("Niv_Gr_3")

Para ver todas las variables:


In [ ]:
# df_all.filter(pl.col("Nombre") == "EC-TCR-01").select(pl.all().is_not_null())
df_all.filter(pl.col("Nombre") == "EC-TCR-01")[:, df_all.select(pl.all().is_not_null().all()).row(0)]

Resultados de una sola variable:


In [ ]:
Id = 97
df = get_df(Id)
# get_plot(Id)
df

#### TCN = Tipo de Cambio Nominal


In [ ]:
res_nivel_3("EC","TCN","Niv_Gr_3")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EC","TCN",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_1","Niv_Descr_2").sort("Niv_Descr_2")

Resultados de una sola variable:


In [ ]:
Id = 619
df = get_df(Id)
# get_plot(Id)
df

### EP = EStadísticas de Precios


In [ ]:
res_nivel_2("EP")

#### IPC  Índice de Precios al Consumidor


In [ ]:
res_nivel_3("EP","IPC","Niv_Gr_3")

Para ver todas las variables:


In [ ]:
res_nivel_7(
    "EP","IPC",
    "Niv_Gr_3","01",
    "Id","Nombre","Niv_Descr_2","Niv_Descr_4").sort("Niv_Descr_2")

Resultados de una sola variable:


In [ ]:
Id = 608
df = get_df(Id)
get_plot(Id)
df

## Nombre

El campo "Nombre" es un código que contiene hasta 8 niveles de agrupación (tomando en cuenta una separación por "-"), siendo los primeros 7 iguales a las definiciones del campo "Grupo".


In [ ]:
df_all.select(
    "Niv_Gr_1","Niv_Gr_2","Nombre",
    "Niv_Descr_1","Niv_Descr_2","Niv_Descr_3","Niv_Descr_4","Niv_Descr_5","Niv_Descr_6","Niv_Descr_7","Niv_Descr_8").sort("Niv_Gr_1")

## Periodicidad


In [ ]:
#| label: tbl-periodicidad
#| tbl-cap: Número de Variables por Periodicidad

# dfmeta = pl.DataFrame(dfmeta)
variable = "Periodicidad"
res = df_all.group_by(variable
    ).agg(
    pl.col("Periodicidad").len().alias("N_Variables"),
    # pl.col("Monto").sum().alias("Millones_USD")
    ).sort("N_Variables",descending=True)
pd.DataFrame(res).to_csv(
        "api/periodicidad.csv",
        sep=";")
print(str(res["N_Variables"].sum()) + " variables")
res

# Graficar una Variable por Id


In [ ]:
Id = 6552
df = get_df(Id)
get_plot(Id)
df.columns
df